In [6]:
import pandas as pd
import csv
import youtokentome as yttm
from torchtext.data import Field, TabularDataset, BucketIterator
import random

In [4]:
data_path = '../../datasets/Machine_translation_EN_RU/data.txt'
en_path = '../../datasets/Machine_translation_EN_RU/en_data.txt'
ru_path = '../../datasets/Machine_translation_EN_RU/ru_data.txt'
en_model_path = 'configs/en_bpe'
ru_model_path = 'configs/ru_bpe'
df = pd.read_csv(data_path, delimiter='\t', header=None)

In [12]:
target = Field(init_token='<sos>',
               eos_token='<eos>',
               lower=True)
source = Field(
    init_token='<sos>',
    eos_token='<eos>',
    lower=True
)

In [13]:
dataset = TabularDataset(
            path=data_path,
            format='tsv',
            fields=[('trg', target), ('src', source)]
        )

In [14]:
random.seed(42)
random_state = random.getstate()
train_data, valid_data, test_data = dataset.split(split_ratio=[0.8, 0.15, 0.05], random_state=random_state)

In [17]:
a = train_data.examples[0]

In [22]:
a.trg

['the', 'apartment', 'is', '1.9', 'km', 'from', 'trinity', 'cathedral.']

In [23]:
ru_data = []
en_data = []
for exm in train_data.examples:
    ru_data.append(' '.join(exm.src))
    en_data.append(' '.join(exm.trg))

In [26]:
with open(ru_path, 'w') as fout:
    fout.write('\n'.join(ru_data))
    
with open(en_path, 'w') as fout:
    fout.write('\n'.join(en_data))

In [27]:
en_bpe = yttm.BPE.train(data=en_path, vocab_size=5000, model=en_model_path)
ru_bpe = yttm.BPE.train(data=ru_path, vocab_size=5000, model=ru_model_path)

In [29]:
ru_test = 'Отель Cordelia расположен в Тбилиси, в 3 минутах ходьбы от Свято-Троицкого собора.'
en_test = 'Cordelia Hotel is situated in Tbilisi, a 3-minute walk away from Saint Trinity Church.'
print(ru_bpe.encode([ru_test.lower()], output_type=yttm.OutputType.SUBWORD))
print(ru_bpe.encode([ru_test.lower()]))

[['▁отель', '▁cor', 'd', 'el', 'ia', '▁расположен', '▁в', '▁тбилиси', ',', '▁в', '▁3', '▁минутах', '▁ходьбы', '▁от', '▁свя', 'то', '-', 'тро', 'и', 'ц', 'кого', '▁собо', 'ра.']]
[[367, 3077, 50, 574, 934, 308, 175, 4025, 30, 175, 426, 378, 532, 190, 1813, 226, 36, 1148, 11, 37, 4128, 1778, 1241]]
